In [1]:
import os
import sys
import cv2
import glob
from time import time
from queue import Queue
from collections import namedtuple

sys.path.append('/home/huy/code/godofeye/lib')

from blueeyes.face_recognition import FaceDetector, FaceRecognition
from blueeyes.utils import Camera

Using TensorFlow backend.


In [2]:
cd ~/Downloads

/home/huy/Downloads


In [3]:
train_set_dict = {}
test_set_dict = {}
for entry in os.scandir('./train_test'):
    id = entry.name
    train_paths = []
    test_paths = []
    all_paths = glob.glob(os.path.join(entry.path, '*'))
    for path in all_paths[0:7]:
        train_paths.append(os.path.abspath(path))
    for path in all_paths[7:10]:
        test_paths.append(os.path.abspath(path))
    train_set_dict[id] = train_paths
    test_set_dict[id] = test_paths

In [4]:
FaceRecognition.train_model(train_set_dict);

[-1.21039182e-01  1.74782827e-01  8.61779079e-02 -3.33773755e-02
 -1.14956424e-01 -6.68598562e-02  3.10725742e-03 -1.77531928e-01
  1.53784081e-01 -7.87654221e-02  2.70476669e-01 -6.92363530e-02
 -2.44772166e-01 -1.14607178e-01 -2.03426406e-02  2.17936695e-01
 -1.03308216e-01 -1.29526660e-01 -2.80359685e-02  3.26068327e-02
  1.55084148e-01 -4.14064489e-02  3.71148065e-02  9.75727588e-02
 -7.69774392e-02 -3.83865774e-01 -1.16470128e-01 -6.38880804e-02
  4.39861696e-03 -6.42535686e-02 -1.11548848e-01  7.67706484e-02
 -1.57672450e-01 -1.16219111e-01  2.80434098e-02  5.20803183e-02
 -1.86638534e-02 -7.68031105e-02  1.72073737e-01  1.49217658e-02
 -2.53471762e-01  7.48784840e-02  3.08252294e-02  2.55183339e-01
  1.69479325e-01  2.12321766e-02  1.10627979e-01 -1.30279362e-01
  9.56937820e-02 -1.49741501e-01  1.99777223e-02  1.12646624e-01
  1.13491572e-01  7.54791824e-03  4.34023663e-02 -1.56479552e-01
 -2.09645499e-02  7.81535879e-02 -1.79408610e-01  7.87970086e-04
  1.19624674e-01 -9.38439

In [5]:
detector = FaceDetector('mtcnn', min_face_size=50)
recog = FaceRecognition(
    model_dir='/home/huy/Downloads', 
    dataset='/home/huy/output',
    vggface=False, 
    # retrain=False
)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Set config.gpu_options.allow_growth to True



In [19]:
TP_count = 0
UNK_count = 0
num_samples = 0
for id, img_paths in test_set_dict.items():
    for path in img_paths:
        img = cv2.imread(path, 1)
        predict_id = recog.recog(img,[[0,0,img.shape[1],img.shape[0]]], threshold=0.38)
        predict_id = predict_id[0][0].split('\n')[0]
#         print(predict_id)
        if predict_id == id:
            TP_count += 1
        elif predict_id == 'unknown':
            UNK_count +=1
        num_samples += 1

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
plt.imshow(img)

In [20]:
TP_count/(num_samples-UNK_count)

0.9682539682539683

In [24]:
# False rate
1 - (TP_count+UNK_count)/num_samples

0.011904761904761862

In [26]:
UNK_count/num_samples

0.625

In [27]:
num_samples, TP_count, UNK_count

(168, 61, 105)